# GPT

In [ ]:
def gpt_analyze_input(user_input, current_step, task):
    """
    Analyze user input using GPT based on the current step and task.

    Parameters:
    user_input (str): The input from the user.
    current_step (str): The current step in the conversation.
    task (str): Specific task for GPT (e.g., "detect_lock_location", "analyze_weather").

    Returns:
    dict: A dictionary containing detected information, or a boolean result for lock location.
    """
    if task == "detect_lock_location":
        prompt = f"Based on the user's input: '{user_input}', does the user want to lock in the location?"
    elif task == "analyze_weather":
        prompt = f"Given the travel dates and location, '{user_input}', will the weather be bad for the trip?"
    else:
        prompt = f"Analyze this user input: '{user_input}' during step '{current_step}' and extract relevant details (e.g., dates, interests, etc.)."

    response = openai.Completion.create(
        engine="gpt-4o-mini",
        prompt=prompt,
        max_tokens=150,
        temperature=0.5
    )
    
    # Example response structure: Let's assume GPT returns a structured result
    gpt_response = response.choices[0].text.strip()

    # Return boolean if task is detect_lock_location
    if task == "detect_lock_location":
        return "yes" in gpt_response.lower() or "lock" in gpt_response.lower()

    # For other tasks, return a structured response (adjust according to actual output format)
    return gpt_response  # You can further parse it based on task requirements


In [ ]:
def gpt_extract_info(user_input, current_step):
    """
    Extract multiple pieces of information from the user's input.
    For example, dates, interests, locations, and questions.
    """
    prompt = f"Analyze this user input: '{user_input}' and extract relevant information such as travel dates, interests, location information, and any questions."
    response = openai.Completion.create(
        engine="gpt-4o-mini",
        prompt=prompt,
        max_tokens=150,
        temperature=0.5
    )
    # change this structure and set return type as JSON

    # Example structure from GPT: Let's assume it returns structured text like this
    # "Dates: May 15th to May 25th, Interests: beaches, hiking, Location: Puerto Rico"
    gpt_response = response.choices[0].text.strip()

    detected_info = {
        "travel_dates": None,
        "interests": None,
        "suggested_locations": None,
        "questions": None
    }
    
    # Parse the structured response based on how GPT formats its response
    if "Dates" in gpt_response:
        detected_info["travel_dates"] = gpt_response.split("Dates: ")[1].split(",")[0]  # Example parsing
    if "Interests" in gpt_response:
        detected_info["interests"] = gpt_response.split("Interests: ")[1].split(",")[0]
    if "Location" in gpt_response:
        detected_info["suggested_locations"] = gpt_response.split("Location: ")[1].split(",")[0]

    return detected_info


In [ ]:
def is_weather_dependent(location):
    """
    Ask GPT whether a location is weather dependent or not.
    """
    prompt = f"Is the location '{location}' highly dependent on weather conditions? (e.g., outdoor activities, beach, hiking)."
    response = openai.Completion.create(
        engine="gpt-4o-mini",
        prompt=prompt,
        max_tokens=100,
        temperature=0.5
    )
    gpt_response = response.choices[0].text.strip()
    return "yes" in gpt_response.lower()


In [ ]:
def check_weather(location, travel_dates):
    """
    Ask GPT or an API to check the weather for the given location and travel dates.
    """
    prompt = f"Check the weather for {location} on these dates: {travel_dates}. Is it expected to be bad?"
    response = openai.Completion.create(
        engine="gpt-4o-mini",
        prompt=prompt,
        max_tokens=100,
        temperature=0.5
    )
    gpt_response = response.choices[0].text.strip()
    return "bad" in gpt_response.lower() or "rain" in gpt_response.lower()


In [ ]:
def confirm_action(user_input):
    """
    Use GPT to detect if the user confirms or rejects an action.
    For example, locking a location or proceeding with a decision.
    """
    prompt = f"Does the user want to confirm this action based on their input: '{user_input}'?"
    response = openai.Completion.create(
        engine="gpt-4o-mini",
        prompt=prompt,
        max_tokens=50,
        temperature=0.5
    )
    gpt_response = response.choices[0].text.strip()
    return "yes" in gpt_response.lower() or "confirm" in gpt_response.lower()


# Bot

In [2]:
def orchestrator(user_input, current_step, conversation_state):
    """
    Orchestrates the conversation based on the current step and user input.

    Parameters:
    user_input (str): The input from the user.
    current_step (str): The current step in the conversation.

    Returns:
    str: The next step in the conversation.
    """
    # Define steps of conversation (flow)
    if current_step == "start":
        
        # Get user input and check for details
        detected_info = gpt_extract_info(user_input, current_step)          ### fix gpt response format
        if detected_info["travel_dates"] and detected_info["interests"]:  # If both dates and interests are detected
            conversation_state["travel_dates"] = detected_info["dates"]
            conversation_state["interests"] = detected_info["interests"]
            return "suggest_locations"
        elif detected_info["travel_dates"]:  # If only dates are detected
            conversation_state["travel_dates"] = detected_info["travel_dates"]
            return "ask_interests"
        elif detected_info["interests"]:  # If only interests are detected
            conversation_state["interests"] = detected_info["interests"]
        # First step: ask for travel dates
        return "ask_travel_dates"
    
    elif current_step == "received_dates":
        if conversation_state.get("interests"):
            # If interests are already detected, suggest locations
            return "suggest_locations"
        # Next step: ask for interests
        return "ask_interests"
    
    elif current_step == "received_interests":
        # save interests
        detected_info = gpt_extract_info(user_input, current_step)              #########fix gpt response format
        conversation_state["interests"] = detected_info["interests"]

        # Now suggest locations based on interests
        return "suggest_locations"
    ########### adding other steps that chatgpt didnt suggest#################
    elif current_step == "received_location":
        current_location = gpt_extract_info(user_input, current_step)   ##### fix gpt response format
        conversation_state["current_location"] = current_location
        # Answer user questions about location
        return "answer_questions"
    elif current_step == "ask_lock_location":
        #after answering questions, ask if user wants to lock in location

        # use gpt to know if they want to lock in location
        ########### GPT ################
        want_to_lock = True | False #GPT response
        ################################
        want_to_lock = confirm_action(user_input)           #########fix gpt response format

        #we asked if "they want to go there." if they say yes, we lock in location (temporarily)
        
        if want_to_lock: #if they want to go there...
            # check if location is weather dependant (maybe another gpt call and they respond {"wheather dependant": True})
            weather_dependant = True | False #GPT response
            weather_dependant = is_weather_dependent(conversation_state["current_location"])        # fix gpt response format
            if weather_dependant:
                #check weather for date
                bad_weather = True | False #(bad weather/good weather)
                bad_weather = check_weather(conversation_state["current_location"], conversation_state["dates"]) # fix gpt response format
                if bad_weather:
                    return "bad_weather"
            else:
                return "lock_location"
            # return "lock_location?"
        # if they say no, we suggest other locations
        else:
            return "suggest_alternatives" # create new step for this"
    elif current_step == "lock_or_change":
        # there was bad weather and we asked the user if they wanted to lock the location.
        # use gpt to know if they want to lock in location
        ########### GPT ################
        want_to_lock = True | False #GPT response
        ################################
        want_to_lock = confirm_action(user_input)           #########fix gpt response format
        if want_to_lock:
            conversation_state["locked_locations"].append(conversation_state["current_location"])
            return "lock_location"
        else:
            return "suggest_alternatives"

    elif current_step == "suggest_other_locations":
        # suggest other locations
        return "suggest_locations"
    
    elif current_step == "end_or_suggest_alternatives":
        # we asked the user if the would like to go anywhere else
        # use gpt to know if they want to go anywhere else

        ########### GPT ################
        want_to_go = True | False #GPT response
        ################################
        want_to_go = confirm_action(user_input)           #########fix gpt response format
        if want_to_go:
            return "suggest_locations"
        else:
            return "end_conversation"
        
    elif current_step == "return_list_of_locked_locations":
        return "give_list"
    ##############################################
    else:
        # Fall back to default
        return "default_response"



In [3]:
def communicator(orchestrator_action, user_input, conversation_state):
    """
    Communicates with the user based on the orchestrator's action.

    Parameters:
    orchestrator_action (str): The action recommended by the orchestrator.
    user_input (str): The input from the user.

    Returns:
    str: The response to the user.
    """
    response = ""

    if orchestrator_action == "ask_travel_dates":
        # Use GPT-4o-mini to rephrase the question
        response= "Great! When are you planning to travel?"
    elif orchestrator_action == "ask_interests":
        # GPT-4o-mini rephrasing
        response= "What kind of places do you like to visit? Are you interested in beaches, historic sites, or something else?"
    elif orchestrator_action == "suggest_locations":
        # Suggest locations based on interests (USE RAG)
        if not conversation_state.get("suggested_locations"):
            rag_response = db.similarity_search(user_input) | rank_appropriate_locations(user_input) # Placeholder function
            conversation_state["suggested_locations"] = rag_response[].   # save suggested locations. Use metadata (landmarks and name)
        else:
            #get index in which current location is in suggested locations
            index = conversation_state["suggested_locations"].index(conversation_state["current_location"])
            # delete current location from suggested locations
            conversation_state["suggested_locations"].pop(index)
            #remove element in index location from rag response
            rag_response.pop(index)
            
        # gpt call to suggest locations. pass rag_response as context
        response= "How about visiting the Castillo San Felipe del Morro in San Juan? It's a beautiful historic site!"
    
    ################ adding other responses that chatgpt didnt suggest
    elif orchestrator_action == "answer_questions":
        #### give info about the location
        info = find_info_on_location(user_input, conversation_state["current_location"]) # placeholder function. 
        # Answer user questions about location. ASK IF THEY WISH TO GO THERE
        response= "Yes, you can visit the Castillo San Felipe del Morro in San Juan. It's a historic fort that dates back to the 16th century. Would you like to go there?"
    elif orchestrator_action == "bad_weather":
        # Inform user about bad weather
        response= "The weather in San Juan is not great for your travel dates. Do you still want to lock in the location?"
    elif orchestrator_action == "lock_location": # lock location
        # Lock the location
        response= "Got it! I've locked that location for your trip. Would you like to go anywhere else?"
    elif orchestrator_action == "suggest_alternatives":
        # Suggest alternative locations
        response = "How about visiting the El Yunque National Forest in Puerto Rico? It's a beautiful rainforest with amazing hiking trails!"
    elif orchestrator_action == "end_conversation":
        # End the conversation
        response= "Great! Enjoy your trip! *wait for list of locked locations*"
    
    elif orchestrator_action == "give_list":
        # Give list of locked locations
        # use conversation_state["locked_locations"]
        # and use compute_distance_to_list() 
        response= "Here are the locations you've locked in: 1. Castillo San Felipe del Morro in San Juan, Puerto Rico. 2. El Yunque National Forest in Puerto Rico. Enjoy your trip!"
    
    ############################
    
    else:
        # Default fallback
        return "I'm not sure how to respond. Could you clarify?"
    
    conversation_state["messages"].append({"user": user_input, "bot": response})     #### add response to conversation state (fix "bot")

    return response




In [ ]:
def cur_step (orchestrator_action):
     # Update the flow based on the orchestrator's action
    if orchestrator_action == "ask_travel_dates":
        return "received_dates"
    elif orchestrator_action == "ask_interests":
        return "received_interests"
    elif orchestrator_action == "suggest_locations":
        #########################
        return "received_location"
    elif orchestrator_action == "answer_questions":
        return "ask_lock_location"
    elif orchestrator_action == "bad_weather":
        return "lock_or_change"
    elif orchestrator_action == "lock_location":
        return "end_or_suggest_alternatives"

    elif orchestrator_action == "suggest_alternatives":
        return "suggest_other_locations"
    elif orchestrator_action == "end_conversation":
        return "return_list_of_locked_locations"
    elif orchestrator_action == "give_list":  
        return "end"
    return "default_response"

In [ ]:
# Example user input and flow

conversation_state = {
    "travel_dates": None,
    "interests": None,
    "locked_location": [],   # i need to save the coordinates of the locations
    "suggested_locations": [],
    "current_location": None,
    "messages": []  # To store the full conversation history
}
rag_response = None

current_step = "start"
while current_step != "end":
    user_input = input("You: ")  # Get user input
    print(f"> {user_input}")
    orchestrator_action = orchestrator(user_input, current_step, conversation_state)
    response = communicator(orchestrator_action, user_input, conversation_state)
    print(f"Bot: {response}")
    
    # Update the flow based on the orchestrator's action
    if orchestrator_action == "ask_travel_dates":
        current_step = "received_dates"
    elif orchestrator_action == "ask_interests":
        current_step = "received_interests"
    elif orchestrator_action == "suggest_locations":
        #########################
        current_step = "received_location"
    elif orchestrator_action == "answer_questions":
        current_step = "ask_lock_location"
    elif orchestrator_action == "bad_weather":
        current_step = "lock_or_change"
    elif orchestrator_action == "lock_location":
        current_step = "end_or_suggest_alternatives"

    elif orchestrator_action == "suggest_alternatives":
        current_step = "suggest_other_locations"
    elif orchestrator_action == "end_conversation":
        current_step = "return_list_of_locked_locations"
    elif orchestrator_action == "give_list":  
        current_step = "end"